In [75]:
import copy
from itertools import chain
import numpy as np

In [76]:
# board = [[1x1 pieces], [1x2 pieces], [2x1 pieces], [2x2 pieces]] (the lower-left block of each piece)
start_board_state = [[[0, 0], [1, 1], [2, 1], [3, 0]], [[0, 3], [0, 1], [3, 3], [3, 1]], [[1, 2]], [[1, 3]]]

In [92]:
class Node:
    def __init__(self, board_state):
        self.parent = None
        self.board_state = board_state
        self.compressed_board = self.compress(self.board_state)
        self.children = []
        self.add_to_history(self.compressed_board)
    
    def add_to_history(self, compressed_board):
        global history
        history.append(compressed_board)
        return
    
    def compress(self, board_state):
        output = []
        for type in range(len(board_state)):
            type_set = {*()}
            for piece in board_state[type]:
                type_set.add(piece[0]*10 + piece[1])
            output.append(type_set)
        return output
    
    def  generate_children(self):
        for type in range(len(self.board_state)):
            for piece in range(len(self.board_state[type])):
                for direction in [[-1, 0], [0, -1], [0, 1], [1, 0]]:
                    potential_child = self.move_if_possible(type, piece, direction)
                    if potential_child == None:
                        continue
                    if not self.child_duplicate(potential_child):
                        self.children.append(Node(potential_child))
        return
    
    def move_if_possible(self, type, piece, direction):
        board_without_piece = copy.deepcopy(self.board_state)
        del board_without_piece[type][piece]
        moved_piece = [sum(x) for x in zip(copy.deepcopy(self.board_state[type][piece]), direction)]
        expanded_piece = self.expand_piece(type, moved_piece)
        for block in expanded_piece:
            if block[0] < 0 or block[0] > 3 or block[1] < 0 or block[1] > 4:
                return
            for type_wo in board_without_piece:
                for piece_wo in type_wo:
                    if block == piece_wo:
                        return
        expanded_board_wo =[]
        for type_wo in range(len(board_without_piece)):
            for piece_wo in board_without_piece[type_wo]:
                expanded_board_wo.append(self.expand_piece(type_wo, piece_wo))
        all_blocks_wo = list(chain.from_iterable(expanded_board_wo))
        for block in expanded_piece:
            if block in all_blocks_wo:
                return
        output = copy.deepcopy(self.board_state)
        output[type][piece] = moved_piece
        return output
    
    def expand_piece(self, type, piece):
        if type == 0:
            output = [piece]
        elif type == 1:
            output = [piece, [sum(x) for x in zip(piece, [0, 1])]]
        elif type == 2:
            output = [piece, [sum(x) for x in zip(piece, [1, 0])]]
        elif type == 3:
            output = [piece, [sum(x) for x in zip(piece, [1, 0])], 
                      [sum(x) for x in zip(piece, [0, 1])], 
                      [sum(x) for x in zip(piece, [1, 1])]]
        return output
    
    def child_duplicate(self, potential_child):
        global history
        compressed_p_c = self.compress(potential_child)
        if compressed_p_c in history:
            return True
        else:
            return False
    
    def draw_board(self):
        expanded_board = []
        for type in range(len(self.board_state)):
            for piece in self.board_state[type]:
                expanded_board.append(self.expand_piece(type, piece))
        board = np.full((5, 4), ' ')
        for i in range(10):
            # X-Wert
            for x in range(len(expanded_board[i])):
                board[expanded_board[i][x][1]][expanded_board[i][x][0]] = i
        return np.flip(np.fliplr(board))

In [93]:
def parent_pointer(child, parent):
    child.parent = parent

In [94]:
def get_layer(root, depth):
    root = [root]
    for layer in range(depth):
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
            root = new_root
    return root

In [95]:
def get_last_layer(root):
    root = [root]
    depth = 0
    while True:
        new_root = []
        for node in root:    
            for child in node.children:
                new_root.append(child)
        if new_root == []:
            return root
        root = new_root
        depth += 1
        print(depth)

In [96]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.history = []

In [97]:
history = []

root = Node(start_board_state)
i = 0
layer = get_layer(root, 0)
while not layer == []:
    for node in layer:
        node.generate_children()
        for child in node.children:
            parent_pointer(child, node)
    print(i)
    i += 1
    layer = get_layer(root, i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167


In [98]:
solution = []
for i in range(170):
    for node in get_layer(root, i):
        if node.compressed_board[3] == {10}:
            solution.append(node)

In [101]:
drawn_solution = []
parent = solution[0]
while not parent == None:
    drawn_solution.append(parent.draw_board())
    parent = parent.parent
drawn_solution.reverse()

In [110]:
for i in drawn_solution:
    print(i, '\n')

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['5' '8' '8' '7']
 ['5' '1' '2' '7']
 ['0' ' ' ' ' '3']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['5' '8' '8' '7']
 ['5' '1' '2' '7']
 [' ' '0' ' ' '3']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['5' '8' '8' '7']
 ['5' '1' '2' '7']
 [' ' '0' '3' ' ']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 [' ' '8' '8' '7']
 ['5' '1' '2' '7']
 ['5' '0' '3' ' ']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 [' ' '8' '8' ' ']
 ['5' '1' '2' '7']
 ['5' '0' '3' '7']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['8' '8' ' ' ' ']
 ['5' '1' '2' '7']
 ['5' '0' '3' '7']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['8' '8' '2' ' ']
 ['5' '1' ' ' '7']
 ['5' '0' '3' '7']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['8' '8' ' ' '2']
 ['5' '1' ' ' '7']
 ['5' '0' '3' '7']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 [' ' '8' '8' '2']
 ['5' '1' ' ' '7']
 ['5' '0' '3' '7']] 

[['4' '9' '9' '6']
 ['4' '9' '9' '6']
 ['5' '8' '8' '2']
 ['5' '1' ' ' '7']
 [' ' '0' '3' '7']] 

[['4' '9' '9' '6']
 

In [114]:
expanded_board = []
for type in range(len(root.board_state)):
    for piece in root.board_state[type]:
        expanded_board.append(root.expand_piece(type, piece))

In [115]:
expanded_board

[[[0, 0]],
 [[1, 1]],
 [[2, 1]],
 [[3, 0]],
 [[0, 3], [0, 4]],
 [[0, 1], [0, 2]],
 [[3, 3], [3, 4]],
 [[3, 1], [3, 2]],
 [[1, 2], [2, 2]],
 [[1, 3], [2, 3], [1, 4], [2, 4]]]